In [ ]:
import numpy as np
import pandas as pd

from tqdm import tqdm, tnrange, tqdm_notebook
tqdm.pandas(desc="Progress:")

from libmy.gk import *

In [ ]:
# read data
# from Bloomberg
df = pd.read_pickle('./../Data/Raw_Data/USDHKD0608.pkl')

In [ ]:
DIRECTORY_DATA = './../Data/Raw_Data/'
DIRECTORY_CALIBRATION = './../Data/Data_for_Calibration/'

CURRENCIES = ['USDHKD0608'] # CHANGE

MATURITIES = {
              '3M': 0.25, 
              '6M': 0.5, 
              '1Y': 1,
             } 

StartDATE = '1999'
EndDATE = '2020' # CHANGE
ROLL = 60 # rolling std

# Conventions
TEST = [{'S_F' : 'S', 'pips_pa' : 'pips'},
        {'S_F' : 'S', 'pips_pa' : 'pa'},
        {'S_F' : 'F', 'pips_pa' : 'pips'},
        {'S_F' : 'F', 'pips_pa' : 'pa'}]

CURRENCIES_CONVENTIONS = {}
for currency in CURRENCIES:
    for maturity, value in MATURITIES.items():
        tmp_dict = {}
        if value < 1:
            tmp_dict['S_F'] = 'S'
        else:
            tmp_dict['S_F'] = 'F'
        if currency[3:] == 'USD':
            tmp_dict['pips_pa'] = 'pips'
        else:
            tmp_dict['pips_pa'] = 'pa'
        
        # check that the format is right
        if tmp_dict not in TEST:
            print('oups')
        ## Add the conventions in the dictionary
        key = currency + maturity
        CURRENCIES_CONVENTIONS[key] = tmp_dict

print(CURRENCIES_CONVENTIONS)

In [ ]:
# complete the dataframe with the sigmaxP/C and KxP/C
def fun(row):
    S = row['S']
    T = MATURITIES[Maturity]
    Rd = row['Rd' + Maturity]
    Rf = row['Rf' + Maturity]
    SEMP = row['stdlogS']
    SATM = row['ATM' + Maturity]
    S25RR = row['25RR' + Maturity]
    S25BF = row['25BF' + Maturity]
    S10RR = row['10RR' + Maturity]
    S10BF = row['10BF' + Maturity]
    
    # compute the corresponding vols
    S10P = SATM + S10BF - S10RR / 2
    S25P = SATM + S25BF - S25RR / 2
    S25C = SATM + S25BF + S25RR / 2
    S10C = SATM + S10BF + S10RR / 2
    
    # bad data
    test = np.array([S, T, S10P, S25P, SATM, S25C, S10C])
    
    # Put an info column for the data problem:
    if test[test<=0].any():
        KATM, K10P, K25P, K25C, K10C = np.nan, np.nan, np.nan, np.nan, np.nan
    else:
        # Compute the corresponding strikes
        KATM = gk.KATM(S, T, Rd, Rf, SATM) 
        K10P = gk.KDelta(S, T, Rd, Rf, S10P, 0.1, Type = -1)
        K25P = gk.KDelta(S, T, Rd, Rf, S25P, 0.25, Type = -1)
        K25C = gk.KDelta(S, T, Rd, Rf, S25C, 0.25, Type = 1)
        K10C = gk.KDelta(S, T, Rd, Rf, S10C, 0.1, Type = 1)
    test = np.concatenate((test, np.array([KATM, K10P, K25P, K25C, K10C])))
    if test[test <= 0].any():
        print('problem:', row.name, test)
        info = 1
    else:
        info = 0
    # Compute the resulting market strangle price
    return pd.Series({
        'S' : S,
        'T' : T,
        'Rd' : Rd,
        'Rf' : Rf,
        'SEMP' : SEMP,
        'S25RR' : S25RR,
        'S25BF' : S25BF,
        'S10RR' : S10RR,
        'S10BF' : S10BF,
        'S10P' : S10P,
        'S25P' : S25P,
        'SATM' : SATM,
        'S25C' : S25C,
        'S10C' : S10C,
        'K10P': K10P,
        'K25P': K25P,
        'KATM': KATM,
        'K25C' : K25C,
        'K10C' : K10C,
        'Info' : info
    })  

In [ ]:
# Compute the different strikes
# read all the data, proceed to the comutations 
for idx, value in enumerate(CURRENCIES):
    
    filename = DIRECTORY_DATA + CURRENCIES[idx] + '.pkl'
    dftmp = pd.read_pickle(filename)
    dftmp['logS'] = np.log(dftmp['S'])-np.log(dftmp['S'].shift(1))
    dftmp['stdlogS'] = (dftmp['logS'].rolling(ROLL).std()) * np.sqrt(250) ### 1Y=250days
    dftmp= dftmp[StartDATE:EndDATE] # slice for the starting date to ending date
#    dftmp = dftmp[dftmp.index.dayofweek<5]
#    dftmp = dftmp[ROLL - 1:]
    
    # Now take all the maturities and save one file per Maturities with the corresponding strikes.
    for Maturity in MATURITIES:
        tmpfilename_pkl = DIRECTORY_CALIBRATION +CURRENCIES[idx] + '_' + Maturity + '_for_Calibration_Average.pkl'
        tmpfilename_csv = DIRECTORY_CALIBRATION +CURRENCIES[idx] + '_' + Maturity + '_for_Calibration_Average.csv'
        tmp_currency = CURRENCIES[idx]+Maturity
        convention = CURRENCIES_CONVENTIONS[tmp_currency]
        gk = GK(**convention)
        print(tmp_currency, ':', convention)
        gk.Info()
        # take the 3 days average of the vols
        Cols = ['ATM' + Maturity, '25RR' + Maturity, '10RR' + Maturity, '25BF' + Maturity, '10BF' + Maturity, 'Rd' + Maturity, 'Rf' + Maturity] 
        dftmp[Cols]= dftmp[Cols].rolling(3).mean()
        dftmp = dftmp[4:]
#         df = dftmp.progress_apply(fun, axis = 1) ### show the progress
        df = dftmp.apply(fun, axis = 1)
        # save the datas in csv and pkl
        df.to_pickle(tmpfilename_pkl)
        df.to_csv(tmpfilename_csv)